# Branch and Cut
I'll be experimenting with cutting planes on a mixed integer model from [miplib](https://miplib.zib.de), the [markshare2](https://miplib.zib.de/instance_details_markshare2.html) problem.

## Observations
After running several variations of the problem I made a few observations.
- This particular model seems to be relatively easy to get the optimal solution through heuristics/cuts/etc., but just plain needs some time to crank through nodes, reduce the upper bound, and prove optimality.
- This doesn't mean cuts aren't helpful, however, as the run without cuts went through about 1.25 million nodes, while the run with conservative cuts went through about .22 million nodes, and the run with only the flow cover cuts about 0.09 million nodes.
- It's hard to know however, exactly what cut types to use, and at what agression levels as, for example, the aggressive level visited more nodes than the very aggressive level, but took a little longer.
- Setting just the particular cuts that dominate can be a good strategy as it uses the information from the cuts as best it can, but doesn't waste too much time calculating cuts that don't help it any.
- That seems to be the main trade-off of cuts, is that they do help shorten the branch-and-bound search significantly (in terms of number of nodes), but at the expense of each node taking a more significant amount of time.
- Like many things with MIPS, the effect of cuts and different kinds of cuts can be highly problem dependent.

In [48]:
import gurobipy as gp

model = gp.read("neos5.mps")
model.params.TimeLimit = 60


Read MPS format model from file neos5.mps
Reading time = 0.00 seconds
neos5: 63 rows, 63 columns, 2016 nonzeros
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [49]:
### Solve baseline problem to compare to
model.reset()
model.params.Cuts = -1  # Default automatic cut selection
model.optimize()
print(model.Runtime)

Discarded solution information
Parameter Cuts unchanged
   Value: -1  Min: -1  Max: 3  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.00000    0   35   24.00000   13.00000  45.8%     -    0s


In [50]:
model.reset()
model.params.Cuts = 0  # No cuts used at all
model.optimize()
print(model.Runtime)


Discarded solution information
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.00000    0   35   24.00000   13.00000  45.8%  

In [51]:
model.reset()
model.params.Cuts = 1  # Conservative cut generation
model.optimize()
print(model.Runtime)


Discarded solution information
Changed value of parameter Cuts to 1
   Prev: 0  Min: -1  Max: 3  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.00000    0   35   24.00000   13.00000  45.8%   

In [52]:
model.reset()
model.params.Cuts = 2  # Agressive cut generation
model.optimize()
print(model.Runtime)

Discarded solution information
Changed value of parameter Cuts to 2
   Prev: 1  Min: -1  Max: 3  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.00000    0   35   24.00000   13.00000  45.8%   

In [53]:
model.reset()
model.params.Cuts = 3  # Very Agressive cut generation
model.optimize()
print(model.Runtime)

Discarded solution information
Changed value of parameter Cuts to 3
   Prev: 2  Min: -1  Max: 3  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.00000    0   35   24.00000   13.00000  45.8%   

In [54]:
model.reset()
model.params.Cuts = 0  # No cut generation

# I looked at the output of the other methods and determined that Flow Cover and
# MIR cuts seemed to be the most helpful, so I turned off all other cuts and 
# just used those two on their most aggressive settings and it seemed to do well
model.params.FlowCoverCuts = 2
model.params.MIRCuts = 2
model.optimize()
print(model.Runtime)

Discarded solution information
Changed value of parameter Cuts to 0
   Prev: 3  Min: -1  Max: 3  Default: -1
Changed value of parameter FlowCoverCuts to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIRCuts to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Object

In [55]:
model.reset()
model.params.Cuts = 0  # No cut generation

# And trying just Flow Cover as this seems to be the most dominant cut
model.params.MIRCuts = 0
model.params.FlowCoverCuts = 2
model.optimize()
print(model.Runtime)

Discarded solution information
Parameter Cuts unchanged
   Value: 0  Min: -1  Max: 3  Default: -1
Changed value of parameter MIRCuts to 0
   Prev: 2  Min: -1  Max: 2  Default: -1
Parameter FlowCoverCuts unchanged
   Value: 2  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 63 rows, 63 columns and 2016 nonzeros
Model fingerprint: 0x179d5a9a
Variable types: 10 continuous, 53 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 63 rows, 63 columns, 2016 nonzeros
Variable types: 10 continuous, 53 integer (53 binary)

Root relaxation: objective 1.300000e+01, 87 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wo